In [1]:
import joblib
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif,f_classif,chi2
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.tree as tree
import sklearn.ensemble as ensem
from sklearn.model_selection import train_test_split
from IPython.display import display, Markdown
import sklearn.metrics as metric
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import catboost as cb
import warnings
import seaborn as sns
from sklearn import preprocessing
warnings.filterwarnings('ignore')

### Carga de los datos de transacciones

In [2]:
transactions = pd.read_csv("ieee-fraud-detection/train_transaction.csv",header=0)

In [3]:
identities = pd.read_csv("ieee-fraud-detection/train_identity.csv",header=0)

### Transformar y procesado de los datos

In [4]:
from sklearn.preprocessing import OrdinalEncoder
def codificar(df, columna):
    dfNuevo = df.copy()
    # Crear codificador
    if np.isin([columna],df.columns).any() == True:
        encoder = OrdinalEncoder(categories=[df[columna].tolist()])
        encoder.fit(df[[columna]])
        encoder.transform(df[[columna]])
        dfNuevo[columna] = encoder.transform(df[[columna]])
    return dfNuevo

def sustituirAusentes(columnas,df,estrategia):
    for c in columnas:
        if estrategia=='median':
            df[c] = df[c].fillna(df[c].median())
            df[c] = df[c].replace(np.inf,df[c].median())
        elif estrategia=='mean':
            df[c] = df[c].fillna(df[c].mean())
            df[c] = df[c].replace(np.inf,df[c].mean())
        else:
            df[c] = df[c].fillna(0)
            df[c] = df[c].replace(np.inf,0)
    return df

def columnasImportantes(df):
    columnasImp=['TransactionAmt','ProductCD','card1','card2','card3','card4','card5','card6','addr1','addr2','C1','C2','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','V12','V13','V19','V20','V29','V30','V53','V54','V56','V60','V61','V62','V64','V69','V70','V282','V283','V290','V291','V292','V302','V303','V304','V310','V315','V317','id_01','id_02','id_05','id_06','id_11','id_12','id_13','id_15','id_16','id_17','id_19','id_20','id_28','id_29','id_31','id_35','id_36','id_37','id_38','DeviceType']
    dfFin = df[columnasImp]
    return dfFin


def normalizar(df):
    normalizer = preprocessing.MinMaxScaler()
    columnas = df.columns
    dataSetClientesNorm = df.copy()
    #Normalizar columnas no categóricas
    categoricas=['card4','card6','ProductCD','id_12','id_15','id_16','id_28','id_29','id_31','id_35','id_36','id_37','id_38','DeviceType']
    continuas=columnas[np.logical_not(columnas.isin(categoricas))]   
    for c in continuas:
        df[c] = normalizer.fit_transform(pd.DataFrame(df[c]))
    return df  

def transformaciones(dft,dfi):
    #Categorizar transacciones
    transaccionesCategorizado = codificar(dft,'card4')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'card6')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'ProductCD')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M6')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'P_emaildomain')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'R_emaildomain')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M1')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M2')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M3')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M4')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M5')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M7')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M8')
    transaccionesCategorizado = codificar(transaccionesCategorizado,'M9')
    #Categorizar identidades
    identidadesCategorizado = codificar(dfi,'id_12')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_15')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_16')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_23')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_27')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_28')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_29')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_30')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_31')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_33')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_34')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_35')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_36')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_37')
    identidadesCategorizado = codificar(identidadesCategorizado,'id_38')
    identidadesCategorizado = codificar(identidadesCategorizado,'DeviceType')
    identidadesCategorizado = codificar(identidadesCategorizado,'DeviceInfo')    
    
    #Mergear
    dfCompleto = pd.merge(transaccionesCategorizado, identidadesCategorizado, on = 'TransactionID', how = 'left')       
        
    #Atributos más relevantes
    dfCompletoAtr = columnasImportantes(dfCompleto) 
        
    #Valores continuos y categóricos
    columnas=dfCompletoAtr.columns
    categoricas=['card4','card6','ProductCD','id_12','id_15','id_16','id_28','id_29','id_31','id_35','id_36','id_37','id_38','DeviceType']
    continuas=columnas[np.logical_not(columnas.isin(categoricas))]
    
    #Sustituir valores ausentes
    dfCompletoAtrS1=sustituirAusentes(categoricas,dfCompletoAtr,'0')

    #Los valores ausentes de las continuas, se sustituyen por su media
    dfCompletoAtrS2=sustituirAusentes(continuas,dfCompletoAtrS1,'mean')
    
    #Normalizar
    dfNorm=normalizar(dfCompletoAtrS2)
    
    return dfNorm

In [5]:
df = transformaciones(transactions,identities)

In [6]:
df

,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,...,id_19,id_20,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType
0,0.002137,590539.0,0.743044,0.525111,0.381679,590490.0,0.306569,590539.0,0.488636,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000900,590539.0,0.100885,0.608000,0.381679,590539.0,0.014599,590539.0,0.511364,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.001840,590539.0,0.210566,0.780000,0.381679,590535.0,0.481752,590538.0,0.522727,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001558,590539.0,0.984824,0.934000,0.381679,590539.0,0.124088,590538.0,0.854545,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001558,590506.0,0.201023,0.828000,0.381679,590539.0,0.014599,590539.0,0.727273,0.836957,...,0.774081,0.078431,144232.0,144232.0,141698.0,144231.0,144232.0,144232.0,144226.0,144232.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0.001526,590539.0,0.319039,0.525111,0.381679,590535.0,0.919708,590538.0,0.390909,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590536,0.001229,590539.0,0.542883,0.250000,0.381679,590539.0,0.905109,590538.0,0.236364,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590537,0.000961,590539.0,0.634456,0.990000,0.381679,590539.0,0.905109,590538.0,0.297727,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
590538,0.003656,590539.0,0.392389,0.762000,0.381679,590539.0,0.905109,590538.0,0.652273,0.836957,...,0.443307,0.541680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Carga del modelo

In [7]:
f2 = open('modeloCatboost.pkl','rb')
modelo = pickle.load(f2)

### Predicción de las trasacciones aplicando el modelo

In [8]:
datos =  modelo.predict(df)

In [9]:
datos

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Interpretación resultados

In [10]:
dfResultados = pd.merge(transactions, identities, on = 'TransactionID', how = 'left')  

In [11]:
dfResultados['Prediction']=datos

#### No fraudulentas

In [12]:
noFraudulentas = dfResultados[dfResultados['Prediction']==0]
noFraudulentas[['TransactionID','TransactionDT','TransactionAmt','card4']]

,TransactionID,TransactionDT,TransactionAmt,card4
0,2987000,86400,68.50,discover
1,2987001,86401,29.00,mastercard
2,2987002,86469,59.00,visa
3,2987003,86499,50.00,mastercard
4,2987004,86506,50.00,mastercard
...,...,...,...,...
590535,3577535,15811047,49.00,visa
590536,3577536,15811049,39.50,mastercard
590537,3577537,15811079,30.95,mastercard
590538,3577538,15811088,117.00,mastercard


#### Fraudulentas

In [13]:
fraudulentas = dfResultados[dfResultados['Prediction']==1]
fraudulentas[['TransactionID','TransactionDT','TransactionAmt','card4']]

,TransactionID,TransactionDT,TransactionAmt,card4
178,2987178,89372,445.000,visa
190,2987190,89565,32.325,mastercard
193,2987193,89611,28.699,mastercard
203,2987203,89760,445.000,visa
240,2987240,90193,37.098,visa
...,...,...,...,...
590368,3577368,15807677,100.000,american express
590371,3577371,15807740,884.000,visa
590373,3577373,15807773,884.000,visa
590445,3577445,15809497,500.000,visa
